# PETs/TETs – Hyperledger Aries – City (Holder) 🏙️

In [1]:
%%javascript
document.title ='🏙️ City'

<IPython.core.display.Javascript object>

## PART 2: Issue a VC to the City Agent

**What:** Receive and store verifiable credential (VC) from the authority agent (i.e., the issuing authority)

**Why:** To prove that the agent is a City to relying parties (e.g., the city agent)

**How:** <br>
1. [Initiate City's AgentCommunicationManager (ACM)](#1)
2. [Connect with the Authority agent](#2)
3. [Get VC from Authority](#3)


**Accompanying Agents and Notebooks:** 
* Authority 🏛️: `02_issue_VC_city.ipynb`

---

### 0 - Setup
#### 0.1 - Imports

In [ ]:
import os

from aries_cloudcontroller import AriesAgentController

import libs.helpers as helpers
from libs.agent_connection_manager import CredentialHolder

#### 0.2 – Variables

In [ ]:
# Get relevant details from .env file
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

--- 

<a id=1></a>

### 1 – Initiate City Agent
#### 1.1 – Init ACA-PY agent controller

In [ ]:
# Setup
agent_controller = AriesAgentController(admin_url,api_key)
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")

#### 1.2 – Start Webhook Server to enable communication with other agents
@todo: is communication with other agents, or with other docker containers?

In [ ]:
# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

#### 1.3 – Init ACM issuing authority

In [ ]:
# The CredentialHolder registers relevant webhook servers and event listeners
city_agent = CredentialHolder(agent_controller)

---

<a id=2></a>

### 2 – Establish a connection with the Authority agent
A connection with the credential issuer (i.e., the authority agent) must be established before a VC can be received. In this scenario, the City requests a connection with the Authority to be certified as an official city agency. Thus, the City agent sends an invitation to the Authority. In real life, the invitation can be shared via video call, phone call, or E-Mail. In this PoC, this is represented by copy and pasting the invitation into the city' notebooks.

#### 2.1 Create invitation to Authority agent


Several state changes of the connection between the City agent, the inviter (A), and the authority agent, the invitee (B),  are required before successfully establishing a connection: 

| Step | State | Agent | Description | Function/Prompt/Variable |
| --- | --- | --- | --- | --- |
| 1 | invitation-sent | A | A sent an invitation to B | `create_connection_invitation()`
| 2 | invitation-received | B | B receives the invitation of A | Prompt: Paste invitation from A |
| 3 | request-sent | B | B sends B connection request | Prompt: Accept invitation OR `auto_accept=True` |
| 4 | request-received | A | A receives the connection request from B | Prompt: Accept invitation request response OR `auto_accept=True` |
| 5 | response-sent | A | A sends B response to B | - |
| 6 | response-received | B | B receives the response from A | - |
| 7 | active (completed) | A | B pings A to finalize connection | Prompt: Trust ping OR `auto_ping=True` |

In [ ]:
# Variables
alias = None
auto_accept= True
label = None

# Receive connection invitation
connection_id = city_agent.receive_connection_invitation(alias=alias, auto_accept=auto_accept, label=label)


<div style="font-size: 25px"><center><b>Break Point 2</b></center></div>
<div style="font-size: 50px"><center>🏙️ ➡️ 🏛</center></div><br>
<center>Please switch to the Authority agent 🏛: open 02_issue_VC_city.ipynb, and run all cells until Break Point 2. Please paste the invitation in Step 2.1 when prompted. If auto_accept is set to False, enter "yes" when prompted before proceeding.</center>

---

<a id=3></a>
### 3 – Request VC from `Authority` agent
#### 3.1 – Message `Authority` with request

In [ ]:
# Verify inbox
message_ids = city_agent.verify_inbox()

for m_id in message_ids:
    city_agent.get_message(m_id)

#### 3.2 – Send response to authority agent

In [ ]:
# Contact authority agent
basic_message = '{"city": "Berlin", "country": "DE"}'
city_agent.send_message(connection_id, basic_message)

<div style="font-size: 25px"><center><b>Break Point 4</b></center></div>
<div style="font-size: 50px"><center>🏙️ ➡️ 🏛</center></div><br>
<center>Please switch to the Authority agent 🏛: Proceed with Step 3.2.</center>

---

#### 3.3 – Respond to Authority's VC offer

The following `request_vc()` function wraps multiple functionalities. First, the offer of the Authority agent must be identified using a credential_exchange_id generated for it. Then, City can respond to the offer. The detailed steps and state changes are presented in the table below, whereby I is the Issuer (i.e., the Authority agent) and H is the Holder (i.e., the City agent)

| Step | State | Role | Description | Function/Prompt/Variable |
| --- | --- | --- | --- | --- |
| 1 | offer_sent | I | I sends I VC offer with personalized information to H| `offer_vc()` |
| 2 | offer_received | H | H receives offer made by I | - |
| 3 | request_sent | H | Request VC offer | `request_vc()` AND (Prompt: request VC OR `auto_request=True`) |
| 4 | request_received | I | M1's request to get VC was received | - |
| 5 | credential_issued | I | Automatic response to issue credential | - |
| 6 | credential_received | H| H receives VC and is asked to store it | Prompt: Store VC OR `auto_store=True`
| 7 | credential_acked | I / H | Credential was issued and stored | - |

In [ ]:
# Get schema_id of city_schema to find the correct credential_exchange_id
identifiers = helpers.get_identifiers()
schema_id = identifiers["city_schema_identifiers"]["schema_id"]

# Variables
auto_request = False # Authomatically requests VC when offer is found
auto_store = False # Automatically stores VC if issued by Authority

# Request VC
city_agent.request_vc(connection_id, schema_id, auto_request, auto_store)

#### 3.4 – Verify that VC was stored in wallet

In [ ]:
city_agent.get_credentials()

--- 

### 4 – Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()

<div style="font-size: 25px"><center><b>Break Point 4</b></center></div>
<div style="font-size: 50px"><center>🏙️ ➡️ 🏛</center></div><br>
<center><b>Please switch to the Authority agent 🏛: Proceed with Step 4</b></center>

---

## 🔥🔥🔥 You can close this notebook now 🔥🔥🔥